# Python 的 Timsort 讲解与实验

作者：你和我一起 👋  
说明：本 notebook 用“可运行”的方式讲清楚 Timsort 的核心思想（run 发现、`minrun` 规范化、合并栈不变量、galloping）以及稳定性与性能特性。

## 1. Timsort 是什么？
- **来源**：Tim Peters 为 Python 设计（因此叫 *Tim*sort），后被 Java 等语言采纳改造。
- **目标**：充分利用真实数据里常见的“已排序片段”（称为 *run*）。
- **方法拼装**：小段用 *插入排序*，大段之间用 *归并*；当一侧连续胜出时进入 *galloping*（指数+二分加速搬运）。
- **性质**：原地、稳定，最坏 \(O(n\log n)\)，对“近乎有序”的数据可接近 \(O(n)\)。

## 2. 先感受一下：`sorted()` 的稳定性
Timsort 是稳定排序：相等键的相对顺序保持不变。

In [ ]:
data = [('a', 1), ('b', 1), ('c', 1)]
res = sorted(data, key=lambda x: x[1])
res

## 3. 从零写个“教学版 Timsort”
> 注意：这不是 CPython 源码的完整复刻，只是一个可读性优先的 **简化版**，帮助你直观看到关键机制。

In [ ]:
from bisect import bisect_left

def _calc_minrun(n):
    """按 Timsort 规则计算 minrun（介于 32~64）。"""
    r = 0
    while n >= 64:
        r |= n & 1
        n >>= 1
    return n + r

def _binary_insertion_sort(a, lo, hi):
    """对 a[lo:hi] 用二分插入排序。"""
    for i in range(lo + 1, hi):
        x = a[i]
        j = bisect_left(a, x, lo, i)
        a[j+1:i+1] = a[j:i]
        a[j] = x

def _reverse_slice(a, start, end):
    i, j = start, end - 1
    while i < j:
        a[i], a[j] = a[j], a[i]
        i += 1
        j -= 1

def _count_run(a, start, end):
    """返回从 start 开始的自然 run [start, run_end)。若是递减，翻转成递增。"""
    if start + 1 >= end:
        return start + 1
    i = start + 1
    # 判断初始趋势
    increasing = a[i-1] <= a[i]
    while i < end and ((a[i-1] <= a[i]) if increasing else (a[i-1] > a[i])):
        i += 1
    run_end = i
    if not increasing:
        _reverse_slice(a, start, run_end)
    return run_end

def _merge(a, base1, len1, base2, len2, tmp):
    """归并 a[base1:base1+len1] 和 a[base2:base2+len2]，到位；tmp 用作缓冲。包含一个简化的 galloping。"""
    # 将较小段拷贝到 tmp
    if len1 <= len2:
        tmp[:len1] = a[base1:base1+len1]
        i = 0; j = base2; k = base1
        gallop = 0
        while i < len1 and j < base2 + len2:
            if tmp[i] <= a[j]:
                a[k] = tmp[i]; i += 1
                gallop = gallop + 1 if gallop >= 0 else 1
            else:
                a[k] = a[j]; j += 1
                gallop = gallop - 1 if gallop <= 0 else -1
            k += 1
            # 简化版 galloping：一侧连续赢 7 次就成批搬
            if gallop >= 7 and i < len1:
                # 找到 tmp[i:] 中第一个 > a[j] 的位置
                cut = bisect_left(tmp, a[j], i, len1) if j < base2 + len2 else len1
                span = cut - i
                a[k:k+span] = tmp[i:cut]
                k += span; i = cut; gallop = 0
            elif gallop <= -7 and j < base2 + len2:
                # 找到 a[j:] 中第一个 >= tmp[i] 的位置
                # 手写二分避免额外切片
                lo, hi = j, base2 + len2
                x = tmp[i]
                while lo < hi:
                    mid = (lo + hi)//2
                    if a[mid] < x:
                        lo = mid + 1
                    else:
                        hi = mid
                span = lo - j
                a[k:k+span] = a[j:lo]
                k += span; j = lo; gallop = 0
        # 收尾
        if i < len1:
            a[k:k+(len1-i)] = tmp[i:len1]
        # 若 j 剩余则已在原地
    else:
        # 对称实现：将右侧较小段拷到 tmp，从尾部归并（为简洁，这里用正向也可）。
        tmp[:len2] = a[base2:base2+len2]
        i = base1; j = 0; k = base1
        gallop = 0
        while i < base1 + len1 and j < len2:
            if a[i] <= tmp[j]:
                a[k] = a[i]; i += 1
                gallop = gallop + 1 if gallop >= 0 else 1
            else:
                a[k] = tmp[j]; j += 1
                gallop = gallop - 1 if gallop <= 0 else -1
            k += 1
            if gallop >= 7 and i < base1 + len1:
                cut = bisect_left(a, tmp[j], i, base1 + len1) if j < len2 else base1 + len1
                span = cut - i
                a[k:k+span] = a[i:cut]
                k += span; i = cut; gallop = 0
            elif gallop <= -7 and j < len2:
                lo, hi = j, len2
                x = a[i]
                while lo < hi:
                    mid = (lo + hi)//2
                    if tmp[mid] < x:
                        lo = mid + 1
                    else:
                        hi = mid
                span = lo - j
                a[k:k+span] = tmp[j:lo]
                k += span; j = lo; gallop = 0
        if j < len2:
            a[k:k+(len2-j)] = tmp[j:len2]

def timsort_demo(a):
    """教学版 timsort：返回排序后的新列表和 run 列表（用于可视化）。"""
    a = list(a)
    n = len(a)
    if n < 2:
        return a, [(0, n)]
    minrun = _calc_minrun(n)
    runs = []  # (base, length)
    i = 0
    while i < n:
        run_end = _count_run(a, i, n)
        run_len = run_end - i
        if run_len < minrun:
            # 扩充到 minrun 或到数组末尾
            target = min(n, i + minrun)
            _binary_insertion_sort(a, i, target)
            run_end = target
        runs.append((i, run_end - i))
        i = run_end

    # 合并栈：维护不变量，简化版本
    # 为了演示，我们只做“相邻 run 从左到右归并”，并在必要时提前合并更小 run，避免坏形状。
    stack = []
    for r in runs:
        stack.append(r)
        # 尝试维护大小关系（非常简化的规则）
        while True:
            if len(stack) >= 3:
                A = stack[-1][1]; B = stack[-2][1]; C = stack[-3][1]
                if not (B > C and A > B + C):
                    # 合并较小的两段（这里简单地合并栈顶两段）
                    (b1, l1) = stack.pop()
                    (b2, l2) = stack.pop()
                    base1, len1, base2, len2 = b2, l2, b1, l1
                    tmp = [None] * min(len1, len2)
                    _merge(a, base1, len1, base2, len2, tmp)
                    stack.append((base1, len1 + len2))
                else:
                    break
            elif len(stack) == 2:
                # 仅在必要时合并防止极端失衡
                A = stack[-1][1]; B = stack[-2][1]
                if A <= B:
                    (b1, l1) = stack.pop()
                    (b2, l2) = stack.pop()
                    base1, len1, base2, len2 = b2, l2, b1, l1
                    tmp = [None] * min(len1, len2)
                    _merge(a, base1, len1, base2, len2, tmp)
                    stack.append((base1, len1 + len2))
                else:
                    break
            else:
                break
    # 将剩余 run 线性合并
    while len(stack) >= 2:
        (b1, l1) = stack.pop()
        (b2, l2) = stack.pop()
        base1, len1, base2, len2 = b2, l2, b1, l1
        tmp = [None] * min(len1, len2)
        _merge(a, base1, len1, base2, len2, tmp)
        stack.append((base1, len1 + len2))
    return a, runs


## 4. 可视化：自然 run 与 `minrun`
让我们生成一些不同“有序度”的数组，看看被切成了哪些 run。

In [ ]:
import matplotlib.pyplot as plt
random.seed(0)

def show_runs(arr, title):
    arr_copy = list(arr)
    # 探测 run（不做排序）
    runs = []
    i, n = 0, len(arr_copy)
    while i < n:
        j = _count_run(arr_copy, i, n)
        runs.append((i, j - i))
        i = j
    # 绘制 run 长度
    lens = [l for _, l in runs]
    plt.figure()
    plt.bar(range(len(lens)), lens)
    plt.title(title + f" | runs={len(lens)}, avg_len={statistics.mean(lens):.1f}")
    plt.xlabel('Run index')
    plt.ylabel('Run length')
    plt.show()

n = 2048
nearly_sorted = list(range(n))
for _ in range(n//50):  # 轻微扰动
    i = random.randrange(n)
    j = random.randrange(n)
    nearly_sorted[i], nearly_sorted[j] = nearly_sorted[j], nearly_sorted[i]

random_data = list(range(n))
random.shuffle(random_data)

descending_blocks = []
block = list(range(128, 0, -1))
descending_blocks = block * (n//len(block))

show_runs(nearly_sorted, 'Nearly sorted')
show_runs(random_data, 'Random')
show_runs(descending_blocks, 'Many short descending runs')

## 5. 看看教学版 Timsort 的排序效果
对同一份数据，比较 `timsort_demo` 与内置 `sorted()` 的结果是否一致。

In [ ]:
arr = random_data[:]
ours, runs = timsort_demo(arr)
built = sorted(arr)
print('一致性：', ours == built)
print('检测到的 run 数量：', len(runs), '，minrun =', _calc_minrun(len(arr)))
runs[:10]  # 看部分 run

## 6. Galloping 直觉实验
我们构造两个高度偏斜的已排序段合并，观察简化版 galloping 的效果（以比较次数粗略感知）。

In [ ]:
def merge_with_compare_count(a1, a2):
    a = a1 + a2
    base1, len1 = 0, len(a1)
    base2, len2 = len(a1), len(a2)
    tmp = [None] * min(len1, len2)
    # 包装 _merge 统计比较次数（粗略）：通过在 bisect/分支处计数
    global cmp_counter
    cmp_counter = 0
    
    # 临时替代比较运算
    class CmpNum(int):
        def __le__(self, other):
            global cmp_counter
            cmp_counter += 1
            return int(self) <= int(other)
        def __lt__(self, other):
            global cmp_counter
            cmp_counter += 1
            return int(self) < int(other)

    a_cmp = list(map(CmpNum, a))
    _merge(a_cmp, base1, len1, base2, len2, tmp)
    return cmp_counter

left = list(range(0, 10_000, 2))
right = list(range(1, 10_000, 2))
c = merge_with_compare_count(left, right)
c

> 这只是“感觉一下” galloping 在偏斜时减少了逐个比较的次数；CPython 的实现更精致且参数可自适应调整。

## 7. 性能小测：`sorted()` vs. 教学版
注意：我们的教学版注重可读性，性能远不如内置的 C 实现，这里只是感受趋势：
- 对“几乎有序”的数据：`sorted()` 更快且接近线性。
- 对“随机”的数据：仍保持 \(O(n\log n)\)。

In [ ]:
def bench_once(arr):
    t0 = time.perf_counter(); s1 = sorted(arr); t1 = time.perf_counter()
    _ = s1[0] if s1 else None
    b = arr[:]
    t2 = time.perf_counter(); s2, _ = timsort_demo(b); t3 = time.perf_counter()
    return (t1 - t0), (t3 - t2)

sizes = [2_000, 8_000, 32_000]
rows = []
for n in sizes:
    base = list(range(n))
    # 近乎有序：随机交换 n/100 个位置
    a = base[:]
    for _ in range(max(1, n//100)):
        i = random.randrange(n); j = random.randrange(n)
        a[i], a[j] = a[j], a[i]
    rs = base[:]
    random.shuffle(rs)
    t_sorted_near, t_demo_near = bench_once(a)
    t_sorted_rand, t_demo_rand = bench_once(rs)
    rows.append((n, t_sorted_near, t_demo_near, t_sorted_rand, t_demo_rand))
rows

### 把结果画出来（绝对时间，越低越好）

In [ ]:
import matplotlib.pyplot as plt
xs = [r[0] for r in rows]
y1 = [r[1] for r in rows]
y2 = [r[2] for r in rows]
y3 = [r[3] for r in rows]
y4 = [r[4] for r in rows]

plt.figure()
plt.plot(xs, y1, marker='o', label='sorted() nearly-sorted')
plt.plot(xs, y2, marker='o', label='demo nearly-sorted')
plt.plot(xs, y3, marker='o', label='sorted() random')
plt.plot(xs, y4, marker='o', label='demo random')
plt.xlabel('n')
plt.ylabel('seconds')
plt.title('Performance comparison (lower is better)')
plt.legend()
plt.show()

## 8. 多关键字排序技巧（利用稳定性）
稳定性允许我们先按次关键字排序，再按主关键字排序，最终等价于多键排序。

In [ ]:
items = [
    {'name':'alice','age':30,'score':90},
    {'name':'bob','age':25,'score':95},
    {'name':'carol','age':25,'score':87},
]
items.sort(key=lambda x: x['score'])  # 次键
items.sort(key=lambda x: x['age'])    # 主键
items

## 9. 一页小结
- Timsort = 识别 **自然 run** + `minrun` 规范化 + **插入排序**（小段）+ **归并**（大段）+ **galloping**（偏斜快车道）。
- **稳定**，适合多键排序场景；真实世界中常见的“近乎有序”数据上性能极佳。
- 生产中直接用 `sorted()` / `list.sort()`；本 notebook 的教学版仅用于理解原理。